# Part 1) CNN을 이용한 CIFAR10 이미지 분류

## Step 0) **딥러닝 패키지 import**




In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10

import numpy as np 
import matplotlib.pyplot as plt

 - `tensorflow.keras.layers (layers)`: 딥러닝 네트워크를 설계할 때 층(layer) 관련 함수들(예:`Dense`, `Conv2D`, `MaxPooling2D`, `SimpleRNN`, `LSTM`)을 모아놓은 라이브러리

 - `tensorflow.keras.models.Model (Model)`: 생성한 층들을 연합하여 하나의 모델로 구성할 때 사용하는 함수

 - `tensorflow.keras.datasets`: TensorFlow에서 딥러닝 실습을 위해 제공해주는 데이터셋 (예: `mnist`, `cifar10`, `cifar100`, `imdb`)

 - `numpy (np)`: 다차원 데이터 처리를 위한 라이브러리 (참고: `pandas`-2차원 데이터에 특화된 라이브러리)

 - `matplotlib.pyplot (plt)`: 데이터 시각화를 위한 라이브러리

 ## Step 1-1) 데이터 불러오기

 <img src="https://user-images.githubusercontent.com/15958325/63308580-41b7fe80-c32e-11e9-827f-98052675c0ea.png" width="800">
 

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


## Step 0) 데이터 전처리

 - CNN은 이미지 처리에 특화된 인공신경망이기 때문에 **3차원 데이터를 입력**으로 받도록 설계 되어있습니다. 
  - MLP에서처럼 이미지를 1차원으로 **reshape할 필요가 없습니다!**
 
 - 단, 학습의 안정성을 위해 각 픽셀의 값이 0부터 1 사이가 되도록 정규화(normalization)는 합니다.

 - Category의 갯수가 3개 이상인 범주형 데이터에 대해서는 **tf.one_hot** 함수를 이용하여 **one-hot encoding**을 해줍니다.

In [ ]:
x_train_cnn = x_train / 255
x_test_cnn = x_test / 255

In [ ]:
y_train_onehot = tf.squeeze(tf.one_hot(y_train, 10))
y_test_onehot = tf.squeeze(tf.one_hot(y_test, 10))

## Step 1) 네트워크 구조 설계

 - `합성곱(convolution)` 연산

<img src="https://datadiving.dothome.co.kr/Deep%203_6.webp" width=500>
 
 - `Convolution`의 기능
  - `Convolution`은 `filter (kernel)`을 통하여 특정 **특징(feature)의 유무 및 위치**를 식별하는데 특화된 연산입니다.
  - 이러한 이유로 이미지 분류 (image classification) 외에도 `객체 위치 식별 (object localization)`, `객체 탐지 (object detection)`, `이미지 분할 (image segmentation)`과 같은 많은 어플리케이션에서 CNN이 사용되었습니다.

<img src='https://datadiving.dothome.co.kr/Deep%203_7.jpg' width=700>
 
 - CNN에 기반한 이미지 분류 네트워크는 
  - 1) 이미지의 **특징을 추출**하는 **CNN** 파트와 (`Conv2D`, `MaxPooling2D` 이용)
    - **추출하는 특징을 점점 구체화(예: 테두리 -> 눈,코,귀 -> 얼굴)**하기 위하여 **Conv layer 중간 중간에 Pooling layer 추가**
    - 일반적으로 특징이 점점 구체화 될수록 많은 수의 특징 추출, **filter의 갯수 증가**
    - 많은 수의 Conv layer를 사용하기 위하여 **zero padding** 사용
      - **Conv layer를 통과했을 때 이미지의 크기가 줄어드는 것 방지**

  - 2) **추출된 특징을 이용하여 classification을 수행**하는 **MLP** 파트로 나뉩니다.
    - CNN의 출력은 3차원이기 때문에 MLP에 넣어줄 때 **1차원으로 reshape** (`Flatten` 이용) 해주어야 합니다.

<img src="https://datadiving.dothome.co.kr/Deep%203_4.png" width=1100> 

- `layers.Conv2D`
 - `filters`: 총 몇개의 filter를 사용할 것인가 (총 몇개의 특징을 추출할 것인가)
 - `kernel_size`: filter의 size (보통 3*3을 사용)
 - `strides`: 보폭 (보통 1)
 - `padding`: conv layer를 거쳐도 이미지의 크기가 줄어들지 않도록 테두리 부분에 0을 집어넣는 것. (padding="same")
 - `activation`: 활성함수, `relu`를 주로 사용

- `layers.MaxPooling2D`
 - `pool_size`: 하나의 값으로 종합하고 싶은 영역 (보통 2*2를 사용)

- `layers.Flatten`

In [ ]:
## 입력계층
img = layers.Input(shape=(32, 32, 3))

## 특징 추출 파트 - CNN
# conv + pool 계속 반복되는 구조
conv1 = layers.Conv2D(filters=16, kernel_size=3, strides=1, padding="same", activation="relu")(img)  # [32, 32, 16]
pool1 = layers.MaxPooling2D(pool_size=2)(conv1) # [16, 16, 16]

conv2 = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding="same", activation="relu")(pool1) # [16, 16, 32]
pool2 = layers.MaxPooling2D(pool_size=2)(conv2) # [8, 8, 32]

conv3 = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="same", activation="relu")(pool2) # [8, 8, 64]
pool3 = layers.MaxPooling2D(pool_size=2)(conv3) # [4, 4, 64]

conv4 = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu")(pool3) # [4, 4, 128]
pool4 = layers.MaxPooling2D(pool_size=2)(conv4) # [2, 2, 128]
 
## 분류 파트 - MLP
mlp_input = layers.Flatten()(pool4) # 2*2*128
prob = layers.Dense(units=10, activation="softmax")(mlp_input)

## 전체 모델
model = Model(inputs=img, outputs=prob)

 - MLP에서와 마찬가지로 `conv layer`의 수와 `filter (feature map)`의 수는 **실험적**으로 결정되기 때문에 여러 개의 모델을 생성 및 학습하여 성능을 비교해보셔야 합니다.

 - 다음 세 가지 값을 입력으로 받아 그에 해당하는 합성곱 신경망을 생성해주는 함수를 만들어봅시다:
   - 입력 데이터 모양 `input_shape`
   - 결과 데이터 차원 `output_dim`
   - `conv layer`의 `filter` 수를 모아놓은 `num_filters_list` (예: num_filters_list = [16, 'max_pool', 32, 'max_pool', 64, 'max_pool', 128, 'max_pool'])
    - 'max_pool'인 경우 `pooling`을 적용하는 것으로 생각해봅시다.

In [ ]:
def MyModel(input_shape, output_dim, num_filters_list):
  # 입력계층 (Input Layer)
  img = layers.Input(shape=input_shape) # cifar10의 경우는 input_shape=[32, 32, 3]

  # 특징 추출 파트 - CNN
  h = img
  for num_filters in num_filters_list:
    if num_filters == "max_pool": # == -> 같다, = -> 할당
      h = layers.MaxPooling2D(pool_size=(2, 2))(h)
    else:
      h = layers.Conv2D(filters=num_filters, kernel_size=3, strides=1, padding="same", activation="relu")(h)

  # 분류 파트 - MLP
  mlp_input = layers.Flatten()(h)
  prob = layers.Dense(units=output_dim, activation="softmax")(mlp_input)

  # 전체 모델
  return Model(inputs=img, outputs=prob)

 - `num_filters_list=[16, 'max_pool', 32, 'max_pool', 64, 'max_pool', 128, 'max_pool']`인 경우에 네트워크를 생성해봅시다.

In [ ]:
model = MyModel(input_shape=(32, 32, 3), output_dim=10, num_filters_list=[16, 'max_pool', 32, 'max_pool', 64, 'max_pool', 128, 'max_pool'])

- `summary` 함수를 이용하여 딥러닝 네트워크가 의도에 맞게 설계되었는지 확인하실 수 있습니다.

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_8 (Conv2D)           (None, 32, 32, 16)        448       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 16, 16, 16)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 32)        4640      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 8, 8, 64)          1849

 - `batch_size=32`, `epochs=15`로 설정하여 생성한 모델을 학습시켜봅시다.

In [ ]:
# 손실함수(loss), 모델 업데이트 알고리즘(optimizer), 평가지표(metrics) 정의
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 학습
model.fit(x=x_train_cnn, y=y_train_onehot, batch_size=32, epochs=15, verbose=1, validation_data=(x_test_cnn, y_test_onehot))

Epoch 1/15
1563/1563 [==============================] - 8s 5ms/step - loss: 1.4643 - accuracy: 0.4692 - val_loss: 1.1608 - val_accuracy: 0.5878
Epoch 2/15
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0790 - accuracy: 0.6193 - val_loss: 1.0253 - val_accuracy: 0.6433
Epoch 3/15
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9139 - accuracy: 0.6812 - val_loss: 0.9421 - val_accuracy: 0.6671
Epoch 4/15
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8005 - accuracy: 0.7218 - val_loss: 0.8603 - val_accuracy: 0.7002
Epoch 5/15
1563/1563 [==============================] - 8s 5ms/step - loss: 0.7069 - accuracy: 0.7536 - val_loss: 0.8617 - val_accuracy: 0.7066
Epoch 6/15
1563/1563 [==============================] - 7s 5ms/step - loss: 0.6353 - accuracy: 0.7779 - val_loss: 0.8367 - val_accuracy: 0.7171
Epoch 7/15
1563/1563 [==============================] - 7s 5ms/step - loss: 0.5728 - accuracy: 0.8000 - val_loss: 0.8410 - val_accuracy:

 # Part 2) **배치 정규화 (Batch Normalization)**와 **잔차 학습 (Residual Connection)**

<img src='https://datadiving.dothome.co.kr/Deep%204_2.png' border='0'></a>

- **ImageNet Challenge**
    - Image classification 알고리즘의 성능을 평가하는 대회
    - 120만개의 학습 데이터 (training data), 5만개의 검증 데이터 (validation data), 10만개의 테스트 데이터 (test data) 
     - 참고: https://image-net.org/download.php
    - 1000개의 클래스
    - 대회에서 우승한 알고리즘들은 모두 컴퓨터 비전 분야 발전에 큰 역할을 해왔습니다.

<img src="https://datadiving.dothome.co.kr/Deep%203_3.webp" width=800>

  - 2012년 `AlexNet` 이후에는 **CNN 기반 모델이 우승**하였고, 2015년도에는 **사람보다 뛰어난** `ResNet`이 개발되었습니다.

  - 우승한 모델의 깊이(depth)가 점점 깊어지는 것을 확인하실 수 있습니다.

    - **VGG16** (2014 ImageNet Challenge Winner; 16 layers)
  <img src="https://miro.medium.com/max/857/1*AqqArOvacibWqeulyP_-8Q.png">

    - **ResNet** (2015 ImageNet Challenge Winner; upto 152 layers)
    <img src="https://blog.kakaocdn.net/dn/bQfaUX/btqYAtD1KcX/Zdc4DLFzR9SoJYBlO6M1uK/img.png">
  
  - **VGG16**을 모방하여 `num_filters_list=[64, 64, 'max_pool', 128, 128, 'max_pool', 256, 256, 256, 'max_pool', 512, 512, 512, 'max_pool']`에 해당하는 네트워크를 생성해봅시다.

In [ ]:
## 모델 생성
vgg = MyModel(input_shape=(32, 32, 3), output_dim=10, num_filters_list=[64, 64, 'max_pool', 128, 128, 'max_pool', 256, 256, 256, 'max_pool', 512, 512, 512, 'max_pool'])

## 모델 출력
vgg.summary()

 - `batch_size=32`, `epochs=5`로 설정하여 간단히 학습시켜봅시다.

In [ ]:
## 손실함수(loss), 모델 업데이트 알고리즘(optimizer), 평가지표(metrics) 설정
vgg.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## 학습
vgg.fit(x=x_train_cnn, y=y_train_onehot, batch_size=32, epochs=5, verbose=1, validation_data=(x_test_cnn, y_test_onehot))

Epoch 1/5
1563/1563 [==============================] - 33s 20ms/step - loss: 2.3028 - accuracy: 0.0971 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/5
1563/1563 [==============================] - 31s 20ms/step - loss: 2.3028 - accuracy: 0.0998 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 3/5
1563/1563 [==============================] - 31s 20ms/step - loss: 2.3028 - accuracy: 0.0985 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 4/5
1563/1563 [==============================] - 31s 20ms/step - loss: 2.3028 - accuracy: 0.0979 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 5/5
1563/1563 [==============================] - 31s 20ms/step - loss: 2.3028 - accuracy: 0.0993 - val_loss: 2.3026 - val_accuracy: 0.1000


 - **학습이 전혀 안 되고 있습니다...!** 왜 그럴까요...?

 - `역전파(Backpropagation)`: 미분의 연쇄법칙(chain rule)에 근거하여 인공신경망의 가중치와 편차를 업데이트하는 알고리즘.

<img src="https://goldenrabbit.co.kr/wp-content/uploads/2022/08/3-1.jpg" width=900>

 - `Gradient Vanishing Problem`: `Backpropagation`에 의해 gradient가 앞단으로 전파되면서 점점 옅어지게 되어 너무 작아져서 **소멸**하게 되는 문제

<img src="https://t1.daumcdn.net/cfile/tistory/997E1B4C5BB6EAF239" width=700>

<img src="https://miro.medium.com/max/1400/1*0yhJ7DbhOX-tRUseljjYoA.png" width=700>

 - `Internal Covariate Shift`
 <img src='https://datadiving.dothome.co.kr/Deep%204_internal%20Covariate%20Shift%202.jpg' width=700>

 <img src="https://gaussian37.github.io/assets/img/dl/concept/batchnorm/3.png" width=700>

 - `배치 정규화 (Batch Normalization)`: 각 레이어를 통과할 때마다 **정규화 (normalization)**하는 레이어를 두어 분포가 변형되지 않도록 조절하는 메커니즘

  - 배치 단위로 실시하기 때문에 `batch normalization`이라고 부릅니다.

 <img src="https://gaussian37.github.io/assets/img/dl/concept/batchnorm/4.png" width=700>


 <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcFYkLE%2FbtqEcUnlXKy%2FZbGZNjObjo2gL2xss8zYzk%2Fimg.png" width=500>

 - 입력계층에서는 데이터 전처리를 통해 `예쁜 데이터`를 넣어준다 하더라도, 레이어를 거듭할수록 데이터의 분포가 변화하게 됨. 
 
 - 운이 나쁘게 음수만 되도록 변화하면, gradient=0이 되어 학습이 되지 않는 문제가 발생

 - 나쁜 분포의 데이터가 relu를 통과하면 학습이 잘 일어나지 않는다. -> **relu를 통과시키기 전**에 분포를 예쁘게 만들어준다 (batch normalization을 적용해준다)! 

- **VGG16**에 `BatchNormalization`을 추가한 네트워크를 생성해봅시다.

In [ ]:
def MyModel(input_shape, output_dim, num_filters_list):
  # 입력계층 (Input Layer)
  img = layers.Input(shape=input_shape) # cifar10의 경우는 input_shape=[32, 32, 3]

  # 특징 추출 파트 - CNN
  h = img
  for num_filters in num_filters_list:
    if num_filters == "max_pool": # == -> 같다, = -> 할당
      h = layers.MaxPooling2D(pool_size=(2, 2))(h)
    else:
      # convolution -> activation
      # convolution -> batch normalization -> activation
      h = layers.Conv2D(filters=num_filters, kernel_size=3, strides=1, padding="same")(h) # convolution
      h = layers.BatchNormalization()(h) # batch normalization
      h = layers.ReLU()(h) # activation

  # 분류 파트 - MLP
  mlp_input = layers.Flatten()(h)
  prob = layers.Dense(units=output_dim, activation="softmax")(mlp_input)

  # 전체 모델
  return Model(inputs=img, outputs=prob)

In [ ]:
def MyModel(input_shape, output_dim, num_filters_list, use_batch_norm):
  # 입력계층 (Input Layer)
  img = layers.Input(shape=input_shape) # cifar10의 경우는 input_shape=[32, 32, 3]

  # 특징 추출 파트 - CNN
  h = img
  for num_filters in num_filters_list:
    if num_filters == "max_pool": # == -> 같다, = -> 할당
      h = layers.MaxPooling2D(pool_size=(2, 2))(h)
    else:
      # convolution -> activation
      # convolution -> batch normalization -> activation
      h = layers.Conv2D(filters=num_filters, kernel_size=3, strides=1, padding="same")(h) # convolution
      if use_batch_norm == True:
        h = layers.BatchNormalization()(h) # batch normalization
      h = layers.ReLU()(h) # activation

  # 분류 파트 - MLP
  mlp_input = layers.Flatten()(h)
  prob = layers.Dense(units=output_dim, activation="softmax")(mlp_input)

  # 전체 모델
  return Model(inputs=img, outputs=prob)

In [ ]:
model = MyModel(input_shape=(32, 32, 3), output_dim=10, num_filters_list=[64, 64, 'max_pool', 128, 128, 'max_pool', 256, 256, 256, 'max_pool', 512, 512, 512, 'max_pool'])
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32, 32, 64)        0         
                                                                 
 conv2d_23 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 64)       256       
 hNormalization)                                           

 - `batch_size=32`, `epochs=30`으로 설정하여 생성한 모델을 학습시켜봅시다.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train_cnn, y_train_onehot, batch_size=32, epochs=30, verbose=1, validation_data=[x_test_cnn, y_test_onehot])

Epoch 1/30
1563/1563 [==============================] - 36s 22ms/step - loss: 1.6284 - accuracy: 0.4203 - val_loss: 1.5531 - val_accuracy: 0.4937
Epoch 2/30
1563/1563 [==============================] - 34s 22ms/step - loss: 0.9541 - accuracy: 0.6682 - val_loss: 1.0486 - val_accuracy: 0.6502
Epoch 3/30
1563/1563 [==============================] - 34s 22ms/step - loss: 0.7223 - accuracy: 0.7533 - val_loss: 1.0954 - val_accuracy: 0.6628
Epoch 4/30
1563/1563 [==============================] - 34s 22ms/step - loss: 0.6008 - accuracy: 0.7947 - val_loss: 0.7124 - val_accuracy: 0.7635
Epoch 5/30
1563/1563 [==============================] - 35s 22ms/step - loss: 0.4969 - accuracy: 0.8318 - val_loss: 0.7029 - val_accuracy: 0.7671
Epoch 6/30
1563/1563 [==============================] - 36s 23ms/step - loss: 0.4031 - accuracy: 0.8630 - val_loss: 0.7336 - val_accuracy: 0.7571
Epoch 7/30
1563/1563 [==============================] - 35s 22ms/step - loss: 0.3322 - accuracy: 0.8851 - val_loss: 0.5781 -

In [ ]:
model.fit(x_train_cnn, y_train_onehot, batch_size=32, epochs=30, verbose=1, validation_data=[x_test_cnn, y_test_onehot])

Epoch 1/30
1563/1563 [==============================] - 49s 22ms/step - loss: 1.4106 - accuracy: 0.4946 - val_loss: 1.4543 - val_accuracy: 0.5539
Epoch 2/30
1563/1563 [==============================] - 35s 22ms/step - loss: 0.8613 - accuracy: 0.7030 - val_loss: 0.9138 - val_accuracy: 0.6924
Epoch 3/30
1563/1563 [==============================] - 34s 22ms/step - loss: 0.6724 - accuracy: 0.7721 - val_loss: 0.8301 - val_accuracy: 0.7174
Epoch 4/30
1563/1563 [==============================] - 35s 22ms/step - loss: 0.5504 - accuracy: 0.8129 - val_loss: 0.9234 - val_accuracy: 0.7190
Epoch 5/30
1563/1563 [==============================] - 34s 22ms/step - loss: 0.4608 - accuracy: 0.8446 - val_loss: 0.5828 - val_accuracy: 0.8100
Epoch 6/30
1563/1563 [==============================] - 35s 22ms/step - loss: 0.3793 - accuracy: 0.8704 - val_loss: 0.6074 - val_accuracy: 0.7949
Epoch 7/30
1563/1563 [==============================] - 34s 22ms/step - loss: 0.3046 - accuracy: 0.8975 - val_loss: 0.7163 -